# **<mark>SAD 2025 - projekt zaliczeniowy</mark>**
### Studia podyplomowe
### Sztuczna Inteligencja i automatyzacja procesów biznesowych w ujęciu praktycznym
### Wykonali: **Aleksander Pawlak, Bartosz Giernalczyk**

# <mark>**Wprowadzenie**</mark>
#### **Na jaki zbiór danych patrzysz?**

Lokalizacja zbioru danych na Kaggle:
https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package/data

Zbiór danych nazywa się „**Rain in Australia**”, znany też jako **weatherAUS.csv**.
<p>WeatherAUS to dziesięcioletnia (lata 2007-2017) seria dziennych obserwacji pogody z wielu stacji meteorologicznych w Australii.</p>

#### **Gdzie/jak został utworzony?**
Dane pochodzą z Australian Bureau of Meteorology (http://www.bom.gov.au/climate/data).

#### **Jakie pytania będą zadawane?**
Najczęstsze cele analizy tego zbioru to:

* **Klasyfikacja**: czy jutro będzie padać? (*główna zmienna zależna* **RainTomorrow**, wartości Yes/No)
* **Regresja**: jeśli tak, to jaka ilość opadu? (zmienna RISK_MM)

Można też analizować:

* Zależności między temperaturą, wilgotnością, prędkością i kierunkiem wiatru a opadami.
* Tworzyć modele predykcyjne od drzew decyzyjnych, przez regresję logistyczną, SVM-y, aż po sieci neuronowe.

#### **Na jakie pytania chcemy odpowiedzieć?**

Celem eksploracyjnej analizy danych jest poznanie struktury i zależności występujących w zbiorze `weatherAUS.csv`. Przykładowe pytania, na które postaramy się odpowiedzieć:

1. Jakie czynniki atmosferyczne najbardziej wpływają na wystąpienie deszczu nastepnego dnia (RainTomorrow)?
2. Jak zmieniają się warunki pogodowe w zaleźności od lokalizacji (Location) i pory roku (Date)?
3. Które zmienne pogodowe najczęsciej występują w dni z ekstremalnymi zjawiskami (np. silne podmuchy wiatru lub duże opady)?


Odpowiedzi na te pytania pomogą zrozumieć dane i mogą stanowić podstawę do dalszego modelowania predykcyjnego.

# <mark>**Czyszczenie i porządkowanie danych**</mark>

## **Diagnostyka danych**

In [ ]:
# statystyki danych - liczba wierszy, kolumn, typy danych, zmienna docelowa

import pandas as pd

# Wczytaj dane
df = pd.read_csv('weatherAUS.csv')

# Ogólne informacje o zbiorze
liczba_wierszy = df.shape[0]
liczba_kolumn = df.shape[1]

# Typy danych
typy_danych = df.dtypes
zmienne_numeryczne = typy_danych[typy_danych == 'float64'].index.tolist()
zmienne_tekstowe = typy_danych[typy_danych == 'object'].index.tolist()

# Wyświetlenie wyników
print("Informacje ogólne o zbiorze:")
print(f"Liczba wierszy: {liczba_wierszy:,}")
print(f"Liczba kolumn: {liczba_kolumn}\n")

print("Typy danych:")
print(f"  • {len(zmienne_numeryczne)} zmiennych numerycznych (float64)")
print(f"  • {len(zmienne_tekstowe)} zmiennych tekstowych (object) – np. {', '.join(zmienne_tekstowe[:3])}")

# Zmienna docelowa
print("\nZmienna docelowa (główna zmienna zależna): RainTomorrow")


Informacje ogólne o zbiorze:

Liczba wierszy: **145 460** (każdy wiersz to jedna obserwacja — dzień z danej lokalizacji)

Liczba kolumn: **23**

Typy danych:

* **16** zmiennych numerycznych (float64) – zawierają zmienne ciągłe, takie jak temperatura, opady, wilgotność

* **7** zmiennych tekstowych (object) – zawierają daty, lokalizacje i dane kategoryczne (np. kierunek wiatru)

Zmienna docelowa (główna zmienna zależna): **RainTomorrow**

<p>

Opis kolumn:

**Date, Location** – data pomiaru i lokalizacja.

**MinTemp, MaxTemp, Temp9am, Temp3pm** – temperatury w różnych porach dnia.

**Rainfall** – ilość opadów (mm).

**Evaporation, Sunshine** – dane o parowaniu i nasłonecznieniu.

**WindGustDir, WindGustSpeed, WindDir9am, WindDir3pm, WindSpeed9am, WindSpeed3pm** – kierunek i prędkość wiatru.

**Humidity9am, Humidity3pm** – wilgotność powietrza.

**Pressure9am, Pressure3pm** – ciśnienie atmosferyczne.

**Cloud9am, Cloud3pm** – zachmurzenie (skala 0–8).

**RainToday, RainTomorrow** – czy padało dziś / czy będzie padać jutro (Yes/No).



## **Braki - wizualizacja, omówienie, wzorzec, błędy w danych i ich naprawa, imputacja braków**

In [ ]:
# Tabela "Braki danych – omówienie" i wykres słupkowy:
# Ilość braków i procent braków + wykres

import pandas as pd
import matplotlib.pyplot as plt

# Wczytanie danych
df = pd.read_csv('weatherAUS.csv')  # Upewnij się, że plik znajduje się w katalogu projektu

# Liczenie braków danych
braki = df.isnull().sum()
procent_brakow = (braki / len(df) * 100).round(1)

# Filtrowanie tylko zmiennych z brakami
braki_df = pd.DataFrame({
    'Zmienna': braki.index,
    'Braki': braki.values,
    'Procent braków': procent_brakow.values
})
braki_df = braki_df[braki_df['Braki'] > 0]
braki_df = braki_df.sort_values(by='Braki', ascending=False)

# Formatowanie liczby braków i procentów dla przejrzystości
braki_df['Braki'] = braki_df['Braki'].apply(lambda x: f"{x:,}")
braki_df['Procent braków'] = braki_df['Procent braków'].apply(lambda x: f"~{x:.0f}%" if x >= 1 else "<1.5%")

# Reset indeksu
braki_df = braki_df.reset_index(drop=True)

# Wyświetlenie tabeli w konsoli
print("\n=== Braki danych – omówienie ===")
print(braki_df)

# Obliczanie procentu brakujących danych (do wykresu)
missing_percent = df.isnull().mean().sort_values(ascending=False) * 100

# Tworzenie poziomego wykresu słupkowego
plt.figure(figsize=(10, 8))
missing_percent.sort_values().plot(kind='barh')
plt.title('Procent brakujących danych w kolumnach')
plt.xlabel('Procent braków [%]')
plt.ylabel('Kolumny')
plt.grid(axis='x')
plt.tight_layout()
plt.show()


**Braki danych – omówienie i wizualizacja**

| Zmienna |	Braki	| Procent braków |
| :-----: | :---: | :------------: |
| Sunshine	| 69 835	| ~48%
| Evaporation	| 62 790 | ~43%
| Cloud3pm	| 59 358	| ~41%
| Cloud9am	| 55 888	| ~38%
| Pressure9am/3pm	| ~15 000	| ~10%
| Wind-related (dir/speed)	| ~10 000	| ~6–7%
| Rainfall, RainToday, RainTomorrow	| ~3 000	| ~2%
| MinTemp/MaxTemp/Temp9am	| <2 000	| <1.5%

***

<p>Najwięcej braków mają zmienne związane z:</p>

* nasłonecznieniem (**Sunshine**)
* parowaniem (**Evaporation**)
* zachmurzeniem o godzinie 15 i 9 (**Cloud3pm** i **Cloud9am**)

![](Procent_braków.png)

In [ ]:
# shadowmapa braku danych

import numpy as np
import matplotlib.pyplot as plt

# Tworzymy binarną macierz braków (1 = brak, 0 = obecność)
missing_matrix = df.isnull().astype(int)

# Używamy próbki 1000 losowych wierszy dla przejrzystości wizualizacji
sample = missing_matrix.sample(1000, random_state=42)

# Rysowanie shadowmapy
plt.figure(figsize=(14, 6))
plt.imshow(sample.T, aspect='auto', cmap='gray_r', interpolation='none')
plt.xlabel("Próbka 1000 rekordów")
plt.ylabel("Kolumny")
plt.title("Shadowmapa braków danych (1 = brak)")
plt.yticks(ticks=np.arange(len(sample.columns)), labels=sample.columns)
plt.colorbar(label="Brak danych")
plt.show()


![](Shadowmapa_braków.png)

In [ ]:
# heatmapa braku danych

import seaborn as sns
import matplotlib.pyplot as plt

# Oblicz macierz korelacji braków (czyli współwystępowania braków danych)
missing_corr = df.isnull().corr()

# Wizualizacja jako heatmapa
plt.figure(figsize=(12, 10))
sns.heatmap(missing_corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Heatmapa współwystępowania braków danych")
plt.show()


![](Heatmapa_braków.png)

Powyższa heatmapa pokazuje, jak bardzo braki danych w jednej kolumnie są skorelowane z brakami w innej kolumnie. <br> Wysoka dodatnia wartość (bliska 1) oznacza, że braki często występują razem.

**Wzorzec** - Heatmapa korelacji

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Zamiana wartości tekstowych na liczby (przed analizą korelacji!)
df['RainToday'] = df['RainToday'].map({'Yes': 1, 'No': 0})
df['RainTomorrow'] = df['RainTomorrow'].map({'Yes': 1, 'No': 0})


data = df

# Oblicz korelacje tylko dla kolumn numerycznych
corr_before_imputations = data.corr(numeric_only=True)

# Rysuj mapę korelacji
plt.figure(figsize=(14, 10))
sns.heatmap(corr_before_imputations, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Mapa korelacji")
plt.tight_layout()
plt.show()


![](Heatmapa_korelacji.png)

**Omówienie wzorca**

Poniżej przedstawiono zmienne o najwyższej korelacji z `RainTomorrow`:

1. **Sunshine** → **-0.45** – im mniej słońca, tym większe prawdopodobieństwo deszczu.
2. **Humidity3pm** → **+0.38** – wyższa wilgotność po południu sprzyja opadom.
3. **Cloud3pm** → **+0.38** – większe zachmurzenie po południu to sygnał nadchodzącego deszczu.
4. **RainToday** → **+0.31** – jeśli dziś padało, jutro również może.

**Wniosek**:  

Żadna pojedyncza zmienna nie wykazuje **silnej liniowej korelacji** z `RainTomorrow`. 

Oznacza to, że **prognozowanie opadów wymaga uwzględnienia kombinacji wielu cech**, a nie opierania się na jednej silnej predykcyjnej zmiennej.

**Błędy w danych i ich naprawa**

Największym problemem zbioru są liczne braki dla kilku kolumn oraz potencjalne wartości odstające.

**Imputacja braków**

In [ ]:
df_imputed = df.copy()

# === 1. Usunięcie wierszy z brakami w zmiennych docelowych ===
df_imputed = df_imputed[df_imputed['RainToday'].notna()]
df_imputed = df_imputed[df_imputed['RainTomorrow'].notna()]


# === 2. Imputacja mody lokalnej dla Cloud9am / Cloud3pm ===
for col in ['Cloud9am', 'Cloud3pm']:
    df_imputed[col] = df_imputed.groupby('Location')[col].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
    )

# === 3. Imputacja mediany lokalnej dla zmiennych ciągłych ===
median_local_cols = [
    "Sunshine", "Evaporation", "Pressure9am", "Pressure3pm",
    "WindGustSpeed", "Humidity3pm", "Rainfall", "WindSpeed3pm",
    "Humidity9am", "WindSpeed9am", "MinTemp", "MaxTemp"
]

for col in median_local_cols:
    df_imputed[col] = df_imputed.groupby('Location')[col].transform(
        lambda x: x.fillna(x.median())
    )

# === 4. Imputacja mediany globalnej dla pozostałych numerycznych ===
numeric_cols = df_imputed.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())

# === 5. Imputacja mody globalnej dla zmiennych kategorycznych ===
for col in ['WindGustDir', 'WindDir9am', 'WindDir3pm']:
    mode = df_imputed[col].mode()
    if not mode.empty:
        df_imputed[col] = df_imputed[col].fillna(mode.iloc[0])


In [ ]:
corr_after_imputations = df_imputed.corr(numeric_only=True)['RainTomorrow'].sort_values(key=abs, ascending=False)


### Porównanie korelacji zmiennych z `RainTomorrow` – przed i po aktualizacji danych

| Zmienna         | Korelacja (wcześniej) | Korelacja (teraz) | Zmiana     |
|-----------------|------------------------|--------------------|------------|
| Sunshine        | -0.4508                | -0.3301            | 🔻 osłabła |
| Humidity3pm     | +0.4462                | +0.4425            | 🔻 minimalnie |
| RainToday       | +0.3131                | +0.3131            | ➖ bez zmian |
| Humidity9am     | +0.2572                | +0.2573            | ➖ bez zmian |
| Rainfall        | +0.2390                | +0.2390            | ➖ bez zmian |
| Cloud3pm        | +0.3819                | +0.2378            | 🔻 wyraźnie osłabła |
| Pressure9am     | -0.2464                | -0.2358            | 🔻 lekko osłabła |
| WindGustSpeed   | +0.2340                | +0.2251            | 🔻 lekko osłabła |
| Pressure3pm     | -0.2260                | -0.2167            | 🔻 lekko osłabła |
| Cloud9am        | +0.3174                | +0.2109            | 🔻 znacząco słabsza |
| MaxTemp         | -0.1592                | -0.1593            | ➖ praktycznie bez zmian |
| Evaporation     | -0.1193                | -0.0922            | 🔻 słabsza korelacja |
| WindSpeed9am    | +0.0910                | +0.0895            | ➖ bez większej różnicy |
| WindSpeed3pm    | +0.0878                | +0.0853            | ➖ bez większej różnicy |
| MinTemp         | +0.0839                | +0.0839            | ➖ bez zmian |

### Wnioski dotyczące wpływu imputacji na korelacje z `RainTomorrow`

Większość zmian w korelacjach po imputacji miała charakter kosmetyczny, co świadczy o stabilności zastosowanych metod uzupełniania danych.  
Natomiast dla zmiennych **`Cloud3pm`**, **`Cloud9am`** oraz **`Sunshine`** zaobserwowano wyraźne osłabienie korelacji z cechą docelową `RainTomorrow`.


Może to sugerować, że użyte techniki imputacji (np. **moda lokalna** lub **mediana lokalna**) spłaszczyły istotną zmienność w tych cechach, co osłabiło ich wartość predykcyjną.

**Rekomendacja**:  
Warto rozważyć zastosowanie bardziej zaawansowanych metod imputacji dla tych zmiennych, takich jak:
- **KNN Imputer** (imputacja na podstawie sąsiadujących obserwacji),
- **regresja wielozmienna** (zależna od innych cech pogodowych),
- lub podejście modelowe z wykorzystaniem np. drzew decyzyjnych.


### Omówienie strategii imputacji braków:



W projekcie zastosowano zróżnicowane strategie imputacji braków danych, dopasowane do rodzaju zmiennej i jej charakterystyki:

---

#### 1. **Imputacja mody lokalnej (dla danych skokowych)**
- **Zastosowano dla:** `Cloud9am`, `Cloud3pm`
- **Opis:** Braki uzupełniane najczęściej występującą wartością (`modą`) w obrębie każdej lokalizacji (`Location`).
- **Uzasadnienie:** Zmienne skokowe 0–8 mają charakter dyskretny i lokalna specyfika pogodowa może mieć wpływ.

---

#### 2. **Imputacja mediany lokalnej (dla zmiennych ciągłych)**
- **Zastosowano dla:**
  - `Sunshine`, `Evaporation`, `Pressure9am`, `Pressure3pm`
  - `WindGustSpeed`, `Humidity3pm`, `Rainfall`, `WindSpeed3pm`
  - `Humidity9am`, `WindSpeed9am`, `MinTemp`, `MaxTemp`
- **Opis:** Braki uzupełniane medianą wyliczaną w obrębie lokalizacji (`Location`).
- **Dodatkowo:** Jeśli dla danej lokalizacji nie można wyliczyć mediany, stosowana jest **mediana globalna**.

---

#### 3. **Imputacja mediany globalnej (dla pozostałych kolumn numerycznych)**
- **Zastosowano dla:** wszystkich kolumn liczbowych, które po wcześniejszych krokach nadal zawierały braki.
- **Opis:** Uzupełnienie braków wartością środkową z całej kolumny.

---

#### 4. **Imputacja mody globalnej (dla zmiennych kategorycznych – kierunki wiatru)**
- **Zastosowano dla:** `WindGustDir`, `WindDir9am`, `WindDir3pm`
- **Opis:** Braki wypełniane najczęściej występującą wartością (modą) w całym zbiorze.

---

#### 5. **Usunięcie wierszy z brakami w kluczowych zmiennych**
- **Zastosowano dla:** `RainToday`, `RainTomorrow`
- **Opis:** Wiersze, w których brakuje wartości zmiennych docelowych, zostały całkowicie usunięte z danych.
- **Uzasadnienie:** Są to zmienne istotne dla predykcji – nie można ich imputować bez ryzyka wprowadzenia błędu poznawczego (*data leakage*).




## **Obserwacje odstające (outliers) - analiza**

**Obserwacje odstające - 7 zmiennych tekstowych**

In [ ]:
# Wybór zmiennych tekstowych
categorical_columns = df_imputed.select_dtypes(include='object').columns

# Analiza unikalnych i rzadkich wartości
category_outliers = {}

for col in categorical_columns:
    if df_imputed[col].isnull().all():
        continue  # pomiń kolumny z samymi NaN
    value_counts = df_imputed[col].value_counts(dropna=False)
    rare_values = value_counts[value_counts < 10]  # można zmienić próg np. <5
    category_outliers[col] = {
        'Liczba unikalnych': value_counts.shape[0],
        'Liczba bardzo rzadkich (<10)': rare_values.shape[0],
        'Najrzadsze wartości': rare_values.index.dropna().tolist()
    }

# Konwersja do DataFrame
category_outliers_df = pd.DataFrame.from_dict(category_outliers, orient='index')

# Wyświetlenie
print(category_outliers_df)


Nie znaleziono niezwykle rzadkich wartości (<10 wystąpień) w żadnej zmiennej tekstowej – co oznacza, że:

* Dane kategoryczne są czyste i spójne,

* Nie ma błędnych wpisów typu "WNW " lub "unknown" itp.,

* Brak potrzeby czyszczenia tekstów

**Obserwacje odstające - 16 zmiennych numerycznych**

<mark>**IQR**</mark>

In [ ]:
# tabela z rankingiem liczby wartosci odstających (wraz z %) wg IQR

import pandas as pd
import numpy as np

# Wczytanie danych
df = pd.read_csv('weatherAUS.csv')

# Wybór tylko zmiennych numerycznych
df_numeric = df.select_dtypes(include=['float64'])

# Obliczanie IQR
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = ((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR))).sum()

# Tworzenie DataFrame z wynikami i kolumną procentową
outliers_iqr_df = pd.DataFrame({
    'Zmienna': df_numeric.columns,
    'Liczba odstających (IQR)': outliers_iqr.values,
    '% całości (IQR)': (outliers_iqr.values / len(df) * 100).round(2)
})

# Sortowanie malejąco
outliers_iqr_df_sorted = outliers_iqr_df.sort_values(by='Liczba odstających (IQR)', ascending=False).reset_index(drop=True)

# Wyświetlenie wyniku
print(outliers_iqr_df_sorted)


| Zmienna       |   Liczba odstających (IQR) |   % całości (IQR) |
|:--------------|---------------------------:|------------------:|
| Rainfall      |                      25578 |             17.58 |
| WindGustSpeed |                       3092 |              2.13 |
| WindSpeed3pm  |                       2523 |              1.73 |
| Evaporation   |                       1995 |              1.37 |
| WindSpeed9am  |                       1817 |              1.25 |
| Humidity9am   |                       1425 |              0.98 |
| Pressure9am   |                       1191 |              0.82 |
| Pressure3pm   |                        919 |              0.63 |
| Temp3pm       |                        764 |              0.53 |
| MaxTemp       |                        489 |              0.34 |
| Temp9am       |                        262 |              0.18 |
| MinTemp       |                         54 |              0.04 |
| Sunshine      |                          0 |              0.00 |
| Humidity3pm   |                          0 |              0.00 |
| Cloud3pm      |                          0 |              0.00 |
| Cloud9am      |                          0 |              0.00 |

In [ ]:
# wykres obserwacji odstających wg IQR

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Wczytanie danych
df = pd.read_csv('weatherAUS.csv')
df_numeric = df.select_dtypes(include=['float64'])

# Obliczanie IQR i liczba outlierów
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = ((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR))).sum()

# Przygotowanie danych do wykresu
df_iqr_sorted = pd.DataFrame({
    'Zmienna': df_numeric.columns,
    'Liczba odstających (IQR)': outliers_iqr.values
}).sort_values(by='Liczba odstających (IQR)', ascending=False)

# Wykres poziomy
df_iqr_sorted.plot(
    y='Liczba odstających (IQR)',
    x='Zmienna',
    kind='barh',
    color='orange',
    figsize=(10, 8),
    legend=False,
    title='Liczba obserwacji odstających wg IQR',
    xlabel='Liczba obserwacji odstających'
)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


![](Liczba_wartości_odstających.png)

Najwięcej wartości odstających występuje w kolumnie **Rainfall**, co może świadczyć o dużej zmienności lub obecności ekstremalnych opadów w danych pogodowych.  
Warto zaznaczyć, że w ponad 60% przypadków `Rainfall = 0`, ze względu na brak opadów danego dnia. Taka zależność tłumaczy liczbę odstających obserwacji.  
Pozostałe kolumny, jak **WindGustSpeed**, **WindSpeed3pm** czy **Evaporation**, mają znacznie mniej odstających, co sugeruje bardziej stabilne rozkłady wartości.

<mark>**Z-score >3**</mark>

In [ ]:
# tabela z rankingiem liczby wartosci odstających (wraz z %) wg Z-score >3

import pandas as pd
import numpy as np

# Wczytanie danych
df = pd.read_csv('weatherAUS.csv')

# Wybór tylko zmiennych numerycznych
df_numeric = df.select_dtypes(include=['float64'])

# Obliczanie Z-score
z_scores = np.abs((df_numeric - df_numeric.mean()) / df_numeric.std())
outliers_z = (z_scores > 3).sum()

# Tworzenie DataFrame z wynikami i kolumną procentową
outliers_z_df = pd.DataFrame({
    'Zmienna': df_numeric.columns,
    'Liczba odstających (Z-score)': outliers_z.values,
    '% całości (Z-score)': (outliers_z.values / len(df) * 100).round(2)
})

# Sortowanie malejąco
outliers_z_df_sorted = outliers_z_df.sort_values(by='Liczba odstających (Z-score)', ascending=False).reset_index(drop=True)

# Wyświetlenie wyniku
print(outliers_z_df_sorted)


In [ ]:
# wykres liczby obserwacji odstających wg Z-score >3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Wczytanie danych
df = pd.read_csv('weatherAUS.csv')
df_numeric = df.select_dtypes(include=['float64'])

# Obliczanie Z-score i liczba outlierów
z_scores = np.abs((df_numeric - df_numeric.mean()) / df_numeric.std())
outliers_z = (z_scores > 3).sum()

# Przygotowanie danych do wykresu
df_zscore_sorted = pd.DataFrame({
    'Zmienna': df_numeric.columns,
    'Liczba odstających (Z-score)': outliers_z.values
}).sort_values(by='Liczba odstających (Z-score)', ascending=False)

# Wykres poziomy
df_zscore_sorted.plot(
    y='Liczba odstających (Z-score)',
    x='Zmienna',
    kind='barh',
    figsize=(10, 8),
    legend=False,
    title='Liczba obserwacji odstających wg Z-score (>3)',
    xlabel='Liczba obserwacji odstających'
)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


Najwięcej wartości odstających ponownie występuje w kolumnie Rainfall, choć ich liczba jest znacznie mniejsza niż przy analizie IQR (ok. 1,7% vs 18%). To sugeruje, że rozkład tej zmiennej może być silnie skośny, przez co Z-score — który zakłada rozkład normalny — nie identyfikuje tylu wartości jako odstające.

Kolumny związane z prędkościami wiatru (WindGustSpeed, WindSpeed9am, WindSpeed3pm) także zawierają zauważalną liczbę wartości odstających.

Ogólnie, Z-score identyfikuje znacznie mniej outlierów niż metoda IQR, co jest typowe dla danych o rozkładzie innym niż normalny.

1. **Rainfall (opady deszczu)**

In [ ]:
# box plot i histogram

import pandas as pd
import matplotlib.pyplot as plt

# Wczytanie danych
file_path = 'weatherAUS.csv'
df = pd.read_csv(file_path)

# Konwersja kolumny Rainfall na wartości numeryczne i usunięcie braków
df['Rainfall'] = pd.to_numeric(df['Rainfall'], errors='coerce')
rainfall = df['Rainfall'].dropna()

# Wyznaczenie wartości odstających na podstawie klasycznego IQR
Q1 = rainfall.quantile(0.25)
Q3 = rainfall.quantile(0.75)
IQR = Q3 - Q1
outliers = rainfall[(rainfall < Q1 - 1.5 * IQR) | (rainfall > Q3 + 1.5 * IQR)]

# Tworzenie box plotu
plt.figure(figsize=(10, 6))
plt.boxplot(outliers, vert=False)
plt.title('Box plot wartości odstających - Rainfall')
plt.xlabel('Rainfall')
plt.grid(True)
plt.show()

# Tworzenie histogramu
plt.figure(figsize=(10, 6))
plt.hist(outliers, bins=100)
plt.title('Histogram wartości odstających - Rainfall')
plt.xlabel('Rainfall')
plt.ylabel('Liczba obserwacji')
plt.grid(True)
plt.show()


![](Boxplot_odstających_Rainfall.png)
![](Histogram_odstających_Rainfall.png)

Zmienna "Rainfall" ma zdecydowanie najwięcej wartości odstających – aż **25 578** przypadków, co stanowi około **17,99%** wszystkich niepustych obserwacji. Rozkład tej zmiennej jest silnie skośny prawostronnie, co oznacza, że większość dni jest sucha lub z niewielkimi opadami, a tylko nieliczne mają intensywne deszcze. Takie zachowanie danych jest typowe dla zmiennych opadowych, gdzie ekstremalne wartości są rzadkie, ale bardzo istotne. W analizie danych warto rozważyć przekształcenia (np. logarytmiczne lub kategoryczne), aby ograniczyć wpływ dużych wartości na model. Jednocześnie nie należy ich eliminować – mogą być kluczowe dla wykrywania zjawisk ekstremalnych, jak powodzie.

2. **WindGustSpeed (prędkość podmuchów wiatru)**

In [ ]:
# box plot i histogram

import pandas as pd
import matplotlib.pyplot as plt

# Wczytanie danych
file_path = 'weatherAUS.csv'  # Zamień na pełną ścieżkę pliku jeśli potrzebujesz
df = pd.read_csv(file_path)

# Konwersja kolumny WindGustSpeed na wartości numeryczne i usunięcie braków
df['WindGustSpeed'] = pd.to_numeric(df['WindGustSpeed'], errors='coerce')
wind_gust = df['WindGustSpeed'].dropna()

# Wyznaczenie wartości odstających metodą IQR
Q1_wind = wind_gust.quantile(0.25)
Q3_wind = wind_gust.quantile(0.75)
IQR_wind = Q3_wind - Q1_wind
outliers_wind = wind_gust[(wind_gust < Q1_wind - 1.5 * IQR_wind) | (wind_gust > Q3_wind + 1.5 * IQR_wind)]

# Tworzenie box plotu
plt.figure(figsize=(10, 6))
plt.boxplot(outliers_wind, vert=False)
plt.title('Box plot wartości odstających - WindGustSpeed')
plt.xlabel('WindGustSpeed')
plt.grid(True)
plt.show()

# Tworzenie histogramu
plt.figure(figsize=(10, 6))
plt.hist(outliers_wind, bins=50)
plt.title('Histogram wartości odstających - WindGustSpeed')
plt.xlabel('WindGustSpeed')
plt.ylabel('Liczba obserwacji')
plt.grid(True)
plt.show()


![](Boxplot_odstających_WindGustSpeed.png)
![](Histogram_odstających_WindGustSpeed.png)

W zmiennej "WindGustSpeed" zidentyfikowano **3 092** wartości odstające, co stanowi około **2,29%** wszystkich obserwacji. Rozkład jest umiarkowanie skośny w prawo, z dużą liczbą umiarkowanych wartości i rzadkimi, bardzo wysokimi porywami wiatru. Wysokie wartości odpowiadają zazwyczaj dynamicznym warunkom atmosferycznym, takim jak burze lub fronty atmosferyczne. Mimo że te dane są technicznie odstające, niekoniecznie powinny być traktowane jako błędy – często są one bardzo cenne w kontekście przewidywania zjawisk ekstremalnych lub zagrożeń pogodowych.

3. **WindSpeed3pm (prędkość wiatru o godzinie 15)**

In [ ]:
# box plot i histogram

import pandas as pd
import matplotlib.pyplot as plt

# Wczytanie danych
file_path = 'weatherAUS.csv'  # Zmień ścieżkę, jeśli plik znajduje się gdzie indziej
df = pd.read_csv(file_path)

# Konwersja kolumny WindSpeed3pm na wartości numeryczne i usunięcie braków
df['WindSpeed3pm'] = pd.to_numeric(df['WindSpeed3pm'], errors='coerce')
wind_speed_3pm = df['WindSpeed3pm'].dropna()

# Obliczenie wartości odstających metodą IQR
Q1 = wind_speed_3pm.quantile(0.25)
Q3 = wind_speed_3pm.quantile(0.75)
IQR = Q3 - Q1
outliers = wind_speed_3pm[(wind_speed_3pm < Q1 - 1.5 * IQR) | (wind_speed_3pm > Q3 + 1.5 * IQR)]

# Tworzenie box plotu
plt.figure(figsize=(10, 6))
plt.boxplot(outliers, vert=False)
plt.title('Box plot wartości odstających - WindSpeed3pm')
plt.xlabel('WindSpeed3pm')
plt.grid(True)
plt.show()

# Tworzenie histogramu
plt.figure(figsize=(10, 6))
plt.hist(outliers, bins=50)
plt.title('Histogram wartości odstających - WindSpeed3pm')
plt.xlabel('WindSpeed3pm')
plt.ylabel('Liczba obserwacji')
plt.grid(True)
plt.show()


![](Boxplot_odstających_WindSpeed3pm.png)
![](Histogram_odstających_WindSpeed3pm.png)

Zmienna "WindSpeed3pm" zawiera **2 523** wartości odstające, czyli około **1,77%** przypadków. Rozkład tej cechy jest lekko skośny w prawo, z dominacją niskich i umiarkowanych wartości, ale z obecnością wyraźnych ekstremów. Odstające wartości mogą odzwierciedlać dynamiczne warunki atmosferyczne, takie jak nagłe zmiany temperatury i ciśnienia, które występują szczególnie w godzinach popołudniowych. W modelach predykcyjnych mogą one wpływać na jakość prognoz, dlatego zaleca się uwzględnienie ich jako potencjalnie informacyjnych, zwłaszcza przy analizach sezonowych lub regionalnych.

4. **Evaporation (parowanie)**

In [ ]:
# box plot i histogram

import pandas as pd
import matplotlib.pyplot as plt

# Wczytanie danych
file_path = 'weatherAUS.csv'  # Zmień ścieżkę pliku, jeśli potrzebne
df = pd.read_csv(file_path)

# Konwersja kolumny Evaporation do typu numerycznego i usunięcie braków
df['Evaporation'] = pd.to_numeric(df['Evaporation'], errors='coerce')
evaporation = df['Evaporation'].dropna()

# Obliczenie wartości odstających przy użyciu metody IQR
Q1 = evaporation.quantile(0.25)
Q3 = evaporation.quantile(0.75)
IQR = Q3 - Q1
outliers = evaporation[(evaporation < Q1 - 1.5 * IQR) | (evaporation > Q3 + 1.5 * IQR)]

# Tworzenie box plotu
plt.figure(figsize=(10, 6))
plt.boxplot(outliers, vert=False)
plt.title('Box plot wartości odstających - Evaporation')
plt.xlabel('Evaporation')
plt.grid(True)
plt.show()

# Tworzenie histogramu
plt.figure(figsize=(10, 6))
plt.hist(outliers, bins=50)
plt.title('Histogram wartości odstających - Evaporation')
plt.xlabel('Evaporation')
plt.ylabel('Liczba obserwacji')
plt.show()


![](Boxplot_odstających_Evaporation.png)
![](Histogram_odstających_Evaporation.png)

Dla zmiennej "Evaporation" występuje **1 995** odstających obserwacji, co daje około **2,41%** wśród niepustych danych. Rozkład tej zmiennej jest skośny prawostronnie, ponieważ parowanie jest zwykle niewielkie, a tylko w gorące, suche dni osiąga wysokie wartości. Odstające punkty mogą być bardzo wartościowe w analizie zjawisk takich jak susze lub intensywne nasłonecznienie. Ze względu na dużą liczbę braków danych w tej kolumnie, należy dokładnie rozważyć, jak te wartości traktować – np. ograniczyć analizę do kompletnego podzbioru lub zastosować techniki imputacji.

5. **WindSpeed9am (prędkość wiatru o godzinie 9)**

In [ ]:
# box plot i histogram

import pandas as pd
import matplotlib.pyplot as plt

# Wczytanie danych
file_path = 'weatherAUS.csv'  # Zmień jeśli plik jest w innej lokalizacji
df = pd.read_csv(file_path)

# WindSpeed9am - przygotowanie danych
df['WindSpeed9am'] = pd.to_numeric(df['WindSpeed9am'], errors='coerce')
wind_speed_9am = df['WindSpeed9am'].dropna()

# Obliczenie wartości odstających (IQR)
Q1 = wind_speed_9am.quantile(0.25)
Q3 = wind_speed_9am.quantile(0.75)
IQR = Q3 - Q1
outliers = wind_speed_9am[(wind_speed_9am < Q1 - 1.5 * IQR) | (wind_speed_9am > Q3 + 1.5 * IQR)]

# Box plot
plt.figure(figsize=(10, 6))
plt.boxplot(outliers, vert=False)
plt.title('Box plot wartości odstających - WindSpeed9am')
plt.xlabel('WindSpeed9am')
plt.grid(True)
plt.show()

# Histogram
plt.figure(figsize=(10, 6))
plt.hist(outliers, bins=50)
plt.title('Histogram wartości odstających - WindSpeed9am')
plt.xlabel('WindSpeed9am')
plt.ylabel('Liczba obserwacji')
plt.grid(True)
plt.show()


![](Boxplot_odstających_WindSpeed9am.png)
![](Histogram_odstających_WindSpeed9am.png)

Zmienna "WindSpeed9am" zawiera **1 817 wartości odstających**, czyli około **1,26%** danych. Jej rozkład jest stosunkowo symetryczny, z lekką skośnością w prawo. Oznacza to, że dane są dość stabilne, ale okazjonalnie pojawiają się wyższe wartości, prawdopodobnie związane z nietypowymi zjawiskami pogodowymi, takimi jak poranne burze lub silne wiatry frontowe. Wartości odstające w tej zmiennej są mniej liczne niż w zmiennych popołudniowych, ale nadal mogą być istotne przy modelowaniu zachowań atmosferycznych w ciągu całego dnia.



* _shadowmapy_
* _heatmapy braków_
* _omówienie wzorca_
* _omówienie strategii imputacji braków_
* _określenie reguł dla danych_
* _wizualizacja naruszeń_
* _naprawa_
* _analiza obserwacji odstających z omówieniem_

# <mark>**Wizualizacje**</mark>

#### **4-5 wykresów przedstawiających określone wzorce lub cechy, które chcesz podkreślić. <br>Każdej wizualizacji powinien towarzyszyć krótki (1-2 zdania) opis tego, co według Ciebie pokazuje**

1. **MinTemp vs MaxTemp**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("weatherAUS.csv")
df = df.dropna(subset=['MinTemp', 'MaxTemp'])

plt.figure(figsize=(6, 4))
plt.scatter(df['MinTemp'], df['MaxTemp'], alpha=0.3)
plt.xlabel('MinTemp (°C)')
plt.ylabel('MaxTemp (°C)')
plt.title('MinTemp vs MaxTemp')
plt.grid(True)
plt.tight_layout()
plt.show()


![](MinTemp_vs_MaxTemp.png)

Wykres pokazuje silną dodatnią korelację — wyższa temperatura minimalna zwykle wiąże się z wyższą temperaturą maksymalną tego dnia.

2. **Rozkład opadów (Rainfall)**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("weatherAUS.csv")
df = df.dropna(subset=['Rainfall'])

plt.figure(figsize=(6, 4))
df['Rainfall'].hist(bins=50)
plt.xlim(0, 20)  # Skupiamy się na wartościach do 20 mm
plt.xlabel('Rainfall (mm)')
plt.ylabel('Liczba dni')
plt.title('Rozkład opadów')
plt.grid(True)
plt.tight_layout()
plt.show()


![](Rozkład_opadów.png)

Większość dni charakteryzuje się bardzo małymi opadami (bliskimi 0 mm), natomiast większe sumy opadów są rzadkie, co wskazuje na ich nieregularny charakter.

3. **Nasłonecznienie a opady następnego dnia (RainTomorrow) (wykres pudełkowy)**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("weatherAUS.csv")
df = df.dropna(subset=['Sunshine', 'RainTomorrow'])
df['RainTomorrow'] = df['RainTomorrow'].map({'No': 0, 'Yes': 1})

plt.figure(figsize=(6, 4))
df.boxplot(column='Sunshine', by='RainTomorrow')
plt.xlabel('Rain Tomorrow (0=No, 1=Yes)')
plt.ylabel('Sunshine (hours)')
plt.title('Sunshine vs RainTomorrow')
plt.suptitle('')
plt.grid(True)
plt.tight_layout()
plt.show()


![](Sunshine_vs_RainTomorrow.png)

Gdy danego dnia świeci słońce przez więcej godzin, szansa na deszcz następnego dnia jest mniejsza — opady częściej występują po dniach z małym nasłonecznieniem.

4. **Wilgotność o 15:00 a opady następnego dnia (RainTomorrow) (wykres pudełkowy)**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("weatherAUS.csv")
df = df.dropna(subset=['Humidity3pm', 'RainTomorrow'])
df['RainTomorrow'] = df['RainTomorrow'].map({'No': 0, 'Yes': 1})

plt.figure(figsize=(6, 4))
df.boxplot(column='Humidity3pm', by='RainTomorrow')
plt.xlabel('Rain Tomorrow (0=No, 1=Yes)')
plt.ylabel('Humidity at 3pm (%)')
plt.title('Humidity3pm vs RainTomorrow')
plt.suptitle('')
plt.grid(True)
plt.tight_layout()
plt.show()

![](Humidity3pm_vs_RainTomorrow.png)

Wyższa wilgotność powietrza po południu koreluje z większym prawdopodobieństwem opadów dnia następnego.

5. **Średnia temperatura maksymalna dla 10 najczęstszych lokalizacji**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("weatherAUS.csv")
df = df.dropna(subset=['Location', 'MaxTemp'])

top_locations = df['Location'].value_counts().head(10).index
avg_max_temp = df[df['Location'].isin(top_locations)].groupby('Location')['MaxTemp'].mean()

plt.figure(figsize=(8, 4))
avg_max_temp.sort_values().plot(kind='bar')
plt.ylabel('Średnia MaxTemp (°C)')
plt.title('Średnia temperatura maksymalna (Top 10 lokalizacji)')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


![](Średnia_temperatura_dla_top10_lokalizacji.png)

Różnice między lokalizacjami są wyraźne — niektóre miejsca są znacznie cieplejsze niż inne, co może mieć związek z położeniem geograficznym i lokalnym klimatem.

# <mark>**Analiza opisowa**</mark>

#### **2-3 analizy opisowe, odnoszące się do pytania badawczego (pytań)** <br> **Każdej analizie powinna towarzyszyć krótka (1-3 zdania) interpretacja**

**1. analiza opisowa - Jakie czynniki atmosferyczne najbardziej wpływają na wystąpienie deszczu nastepnego dnia (RainTomorrow)?**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

top_features = ['Sunshine', 'Humidity3pm', 'RainToday', 'Humidity9am', 'Rainfall', 'Cloud3pm', 'Pressure9am']

# Konfiguracja siatki wykresów: 3 wiersze, 3 kolumny (dostosuj do liczby cech)
n_features = len(top_features)
cols = 3
rows = (n_features + cols - 1) // cols  # zaokrąglenie w górę

fig, axes = plt.subplots(rows, cols, figsize=(15, 4 * rows))
axes = axes.flatten()

# Rysowanie każdego boxplota
for i, feature in enumerate(top_features):
    sns.boxplot(x='RainTomorrow', y=feature, data=df_imputed, ax=axes[i])
    axes[i].set_title(f'{feature} vs RainTomorrow')

# Ukrycie pustych subplotów (jeśli niepełna siatka)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


![](Boxploty_top_korelujących_cech_z_RainTomorrow.png)

Do wcześniej ustalonych cech, które najbardziej korelowały ze zmienną docelową `RainTomorrow`, utworzono boxploty wizualizujące zależności.

Najważniejsze wnioski z wykresów:

- **RainToday** – bardzo silny predyktor; jeśli dziś padało, jutro również istnieje wysokie prawdopodobieństwo opadów.
- **Humidity3pm** – wyższa wilgotność popołudniu istotnie zwiększa szansę na deszcz.
- **Sunshine** – im mniej słońca danego dnia, tym większe prawdopodobieństwo opadów następnego dnia.
- **Cloud3pm** – duże zachmurzenie po południu sprzyja wystąpieniu deszczu.
- **Rainfall** – obecność opadów danego dnia (nawet niewielkich) koreluje z deszczem dnia następnego.
- **Pressure9am** – niższe ciśnienie rano związane jest z większą szansą na deszcz (typowy sygnał niżu atmosferycznego).
- **Humidity9am** – umiarkowany wpływ; może działać wspomagająco w predykcji.

Wizualizacje potwierdzają wyniki analizy korelacji i wspierają wybór tych cech jako istotnych przy budowie modelu predykcyjnego.


**2. analiza opisowa - Jak zmieniają się warunki pogodowe w zaleźności od lokalizacji (Location) i pory roku (Date)?**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Agregacja: tylko średnie opady w lokalizacjach, posortowane malejąco
srednie_opady = df.groupby('Location')['Rainfall'].mean().sort_values(ascending=False)

# Wykres słupkowy
plt.figure(figsize=(14, 6))
sns.barplot(x=srednie_opady.index, y=srednie_opady.values)
plt.xticks(rotation=90)
plt.title('Średnie opady w lokalizacjach (najwyższe → najniższe)')
plt.ylabel('Rainfall [mm]')
plt.xlabel('Location')
plt.tight_layout()
plt.show()


# Upewniamy się, że Date to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Dodanie kolumny z miesiącem
df['Month'] = df['Date'].dt.month

# Agregacja: średnie opady i temperatura maksymalna wg miesiąca
miesieczne_srednie = df.groupby('Month')[['Rainfall', 'MaxTemp']].mean()

# Wykres: średnie opady
plt.figure(figsize=(10, 5))
sns.lineplot(x=miesieczne_srednie.index, y=miesieczne_srednie['Rainfall'], marker='o')
plt.title('Średnie opady deszczu w poszczególnych miesiącach')
plt.xlabel('Miesiąc')
plt.ylabel('Rainfall [mm]')
plt.xticks(ticks=range(1, 13))
plt.grid(True)
plt.tight_layout()
plt.show()

# Wykres: średnia temperatura maksymalna
plt.figure(figsize=(10, 5))
sns.lineplot(x=miesieczne_srednie.index, y=miesieczne_srednie['MaxTemp'], marker='o', color='orange')
plt.title('Średnia temperatura maksymalna w poszczególnych miesiącach')
plt.xlabel('Miesiąc')
plt.ylabel('MaxTemp [°C]')
plt.xticks(ticks=range(1, 13))
plt.grid(True)
plt.tight_layout()
plt.show()


![](Średnie_opady_w_lokalizacjach.png)

Widać wyraźne zróżnicowanie opadów w zależności od lokalizacji.  
Najwięcej deszczu notuje się w Cairns, Darwin i Coffs Harbour, natomiast najmniej - w Uluru, AliceSprings i Woomera.  
Lokalizacje przybrzeżne i tropikalne mają zdecydowanie wyższe średnie opady niż centralne obszary Australii.

![](Średnie_opady_deszczu_po_miesiącach.png)

Temperatura maksymalna osiąga najwyższe wartości w miesiącach letnich (styczeń-luty), a najniższe w zimowych (lipiec).  
Wykres dobrze odzwierciedla cykl sezonowy klimatu australijskiego (z odwróconą porą roku względem półkuli północnej).

![](Średnia_temperatura_maksymalna_po_miesiącach.png)

Największe opady przypadają na luty i czerwiec, natomiast najmniej pada w październiku.  
Sezonowość opadów jest zauważalna, ale nie tak regularna jak w przypadku temperatury - może być zależna od regionalnych różnic klimatycznych.


**3. analiza opisowa - Które zmienne pogodowe najczęsciej występują w dni z ekstremalnymi zjawiskami (np. silne podmuchy wiatru lub duże opady)?**

In [ ]:
# --- 1. Ustalenie progów ekstremalnych zjawisk ---
rain_threshold = df_imputed['Rainfall'].quantile(0.95)
wind_threshold = df_imputed['WindGustSpeed'].quantile(0.95)

# --- 2. Filtrowanie dni ekstremalnych ---
extreme_days = df_imputed[(df_imputed['Rainfall'] > rain_threshold) | 
                          (df_imputed['WindGustSpeed'] > wind_threshold)]

# --- 3. Wybór zmiennych pogodowych do porównania ---
features = ['Humidity3pm', 'Humidity9am', 'Sunshine', 'Cloud3pm', 'Pressure3pm',
            'Pressure9am', 'MinTemp', 'MaxTemp', 'WindSpeed3pm', 'WindSpeed9am']

# --- 4. Obliczenie średnich wartości ---
extreme_means = extreme_days[features].mean()
normal_means = df_imputed[~df_imputed.index.isin(extreme_days.index)][features].mean()

# --- 5. Tabela porównawcza ---
comparison_df = pd.DataFrame({
    'Średnia (ekstremalne dni)': extreme_means,
    'Średnia (normalne dni)': normal_means,
    'Różnica': extreme_means - normal_means
}).sort_values(by='Różnica', ascending=False)

plt.figure(figsize=(12, 6))
comparison_df['Różnica'].sort_values().plot(kind='barh', color='orange')
plt.axvline(0, color='gray', linestyle='--')  # linia odniesienia dla 0
plt.title('Różnice średnich wartości cech w dniach ekstremalnych vs normalnych')
plt.xlabel('Różnica wartości (ekstremalne - normalne)')
plt.tight_layout()
plt.show()


![](Różnice_w_dniach_ekstremalnych.png)

Na podstawie wykresu różnic średnich wartości cech między dniami ekstremalnymi (silny wiatr lub duże opady) a dniami normalnymi, można zauważyć:

- **Większe wartości**: `Humidity3pm`, `Humidity9am`, `WindSpeed3pm`, `WindSpeed9am` — warunki typowe dla niestabilnej, wilgotnej atmosfery i intensywnego ruchu powietrza.
- **Niższe wartości**: `Pressure9am`, `Pressure3pm`, `Sunshine`, `MaxTemp` — spadek ciśnienia i ograniczenie nasłonecznienia, co sprzyja występowaniu burz.

Wniosek: ekstremalne zjawiska pogodowe zachodzą głównie przy wysokiej wilgotności, silnym wietrze i niskim ciśnieniu.


# <mark>**Wnioski**</mark>

* Najsilniejsze zależności z RainTomorrow wykazały zmienne: Sunshine, RainToday, Humidity3pm oraz Cloud 3pm.  
* Boxploty potwierdziły, że deszcz jest bardziej prawdopodobny przy dużej wilgotności, niskim nasłonecznieniu i wcześniejszych opadach.  
* Dane zawierały liczne braki, szczgólnie w Sunshine, Evaporation i Cloud3pm, które uzupełniono z wykorzystaniem mody i mediany lokalnej.
* Zależności pomiędzy zmiennymi po imputacji pozostały w większości stabilne, jednak część cech wykazała pogorszenie korelacji, co sugeruje, aby  
  uzupełnienie tych kolumn zostało wykonane bardziej złożonymi metodami.  
* Analiza sezonowa wykazała, że latem temperatury są najwyższe, a opady najbardziej zróżnicowane. Ponadto potwierdzono, że warunki pogodowe różnią się  
  dla poszczególnych lokalizacji.  
* Dni z ekstremalnymi zjawiskami pogodowymi (silny wiatr, intensywne opadu) cechują się wyższą wilgotnością, a niższym ciśnieniem.  
* Wnioski z wykonanej analizy są spójne z rzeczywistymi zjawiskami meteorologicznymi.
